# Install dependencies
```shell
sudo apt update
sudo apt upgrade
sudo apt install python3 python3-pip

pip install numpy
pip install pandas
pip install sklearn
```

# Init Libraries and Dataset

In [111]:
import numpy as np
import pandas as pd

from sklearn.neighbors import KNeighborsClassifier
# from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

testSet = pd.read_csv('test.csv')
trainSet = pd.read_csv('train.csv')

# Prelim Transformations
This section aims to convert all columns with multiple string values into multiple columns with a single string value
This section then aims to convert the string values of the solution column into proper numbers (1 or 0)

In [112]:


# This prints the columns that need to be mapped and the unique data to remap
# for col in testSet.axes[1]:
#     if testSet[col].dtype == 'object':
#         print(col + ": ", end="")
#         print(pd.unique(testSet[col]))
# print('\n')

# This prints the columns that contain NaN values
# nan_things = trainSet_transformed.isna().any()
# print(nan_things)


# For every text column, break into multiple binary columns
testSet_transformed = testSet
for col in testSet.axes[1]:
    if testSet[col].dtype == 'object' and col != 'satisfaction':
        testSet_transformed = pd.get_dummies(testSet_transformed, prefix=[col], columns=[col])

trainSet_transformed = trainSet
for col in trainSet.axes[1]:
    if trainSet[col].dtype == 'object' and col != 'satisfaction':
        trainSet_transformed = pd.get_dummies(trainSet_transformed, prefix=[col], columns=[col])


# Reorganize the data such that the solution axes is last
solutionAxes = ['satisfaction']
testSet_transformed = testSet_transformed[
    [col for col in testSet_transformed if col not in solutionAxes] + 
    [col for col in solutionAxes if col in testSet_transformed]
]
trainSet_transformed = trainSet_transformed[
    [col for col in trainSet_transformed if col not in solutionAxes] + 
    [col for col in solutionAxes if col in trainSet_transformed]
]

# Replace solutionAxes values of 'satisfied' or 'neutral or dissatisfied' with 1 or 0 respectively
testSet_transformed['satisfaction'] = testSet_transformed['satisfaction'].replace(['satisfied', 'neutral or dissatisfied'], [1, 0])
trainSet_transformed['satisfaction'] = trainSet_transformed['satisfaction'].replace(['satisfied', 'neutral or dissatisfied'], [1, 0])
# print(testSet_transformed['satisfaction'])
# print(testSet_transformed.head())

# print(testSet_transformed.columns, '\n')

# replace all nan values with 0's
# TODO: a better method would be to either manipulate the values or remove the rows 
trainSet_transformed = trainSet_transformed.replace(np.nan, 0)
testSet_transformed = testSet_transformed.replace(np.nan, 0)

# testSet_transformed['Arrival Delay in Minutes'] = testSet_transformed['Arrival Delay in Minutes'].fillna(testSet_transformed['Arrival Delay in Minutes'].median(axis = 0), inplace = True)

# Load the test and train data
testData = testSet_transformed[testSet_transformed.axes[1][2:-1]]
testSolution = testSet_transformed[testSet_transformed.axes[1][-1]]
trainData = trainSet_transformed[trainSet_transformed.axes[1][2:-1]]
trainSolution = trainSet_transformed[trainSet_transformed.axes[1][-1]]

# print(trainData.columns, '\n')
# print(trainSolution, '\n')


In [113]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(trainData, trainSolution)

testPredict = knn.predict(testData)
accuracy = accuracy_score(testSolution,testPredict)
print(accuracy)


0.7498075146288882
